In [1]:
with open("raw-data/topsites.txt") as ts:
    topsites = ts.readlines()
topsites[0:5]

['google.com\n',
 'youtube.com\n',
 'facebook.com\n',
 'reddit.com\n',
 'amazon.com\n']

In [2]:
topsites = [x.strip() for x in topsites]
topsites[0:5]

['google.com', 'youtube.com', 'facebook.com', 'reddit.com', 'amazon.com']

In [3]:
import json
with open("raw-data/product_hunt_urls.json") as ph:
    producthunt = json.load(ph)

In [4]:
producthunt[0:5]

['https://www.global-starters.com',
 'http://getembark.io',
 'https://www.allanswered.com',
 'https://www.stepsapp.xyz',
 'http://senstone.io']

In [5]:
unfiltered_sites = topsites + producthunt

In [6]:
len(unfiltered_sites)

6242

In [9]:
# now let's get all the URLS that I actually have already. 
# this will need to be updated with recent stuff collected by RAs after 11-09-17, but I can pull that from psql.
# I'm going to discard any sites that my former RAs said they couldn't find ToS or other contracts on.  
with open ("raw-data/contracts7-24-17.json") as oldk:
    olddata = [json.loads(x) for x in oldk.readlines()]
    oldurls = []
    for x in olddata:
        if "url" in x:
            oldurls.append(x["url"])

In [10]:
len(oldurls)

339

In [11]:
oldurls[0:5]

['https://hasura.io',
 'https://nightwatch.io',
 'http://www.submarinecablemap.com',
 'http://www.theaivc.com',
 'http://www.slimfoldwallet.com']

In [13]:
# now the more recent data
import pandas as pd
recentdata = pd.read_csv("raw-data/contracts-11-09-17.csv")
recentdata.head()

,id,contracturl,linktext,student,businesscategory,company,contract,noncontractdescription,foundnoncontract,noncontracttext,entered
0,2,https://ankiweb.net/account/terms,Legal,paul,internet companies,Anki,Terms and Conditions\r\nPlease read through th...,None (arbitrary add),f,AnkiWeb is a free companion to the computer ve...,2017-08-17 03:05:43.995519
1,3,https://www.spotify.com/us/legal/end-user-agre...,terms and conditions of use,paul,internet companies,Spotify,Spotify Terms and Conditions of Use\r\n\r\nEff...,None (arbitrary add),f,"About Us\r\nWith Spotify, it’s easy to find th...",2017-08-23 02:48:38.207869
2,4,https://www.clearxchange.com/legal,LEGAL AND PRIVACY,paul,internet companies,ClearXChange,CLEARXCHANGE TERMS OF USE\r\nLast updated: Dec...,None (arbitrary add),f,"clearXchange® is becoming Zelle℠!\r\nJune 12, ...",2017-08-27 20:04:28.021259
3,5,https://www.clearxchange.com/legal,service agreement,paul,internet companies,ClearXChange,THE CLEARXCHANGE.COM USER SERVICE AGREEMENT\r\...,None (arbitrary add),f,"News\r\n\r\nJournalists and analysts, please c...",2017-08-27 20:06:14.873878
4,6,https://www.clearxchange.com/legal#esign,e-sign agreement,paul,internet companies,ClearXChange,CLEARXCHANGE E-SIGN DISCLOSURE AND CONSENT\r\n...,None (arbitrary add),f,HOW IT WORKS\r\nWe unite with the top banks to...,2017-08-27 20:08:03.822403


In [14]:
recenturls = list(recentdata["contracturl"].copy())

In [15]:
recenturls[0:5]

['https://ankiweb.net/account/terms',
 'https://www.spotify.com/us/legal/end-user-agreement/',
 'https://www.clearxchange.com/legal',
 'https://www.clearxchange.com/legal',
 'https://www.clearxchange.com/legal#esign']

In [16]:
import urllib.parse as url
url.urlparse(recenturls[0])

ParseResult(scheme='https', netloc='ankiweb.net', path='/account/terms', params='', query='', fragment='')

In [17]:
url.urlparse(topsites[0])

ParseResult(scheme='', netloc='', path='google.com', params='', query='', fragment='')

In [18]:
url.urlparse(producthunt[0])

ParseResult(scheme='https', netloc='www.global-starters.com', path='', params='', query='', fragment='')

In [24]:
url.urlparse(oldurls[0]).netloc

'hasura.io'

In [25]:
candidate_sites = topsites + [url.urlparse(x).netloc for x in producthunt]

In [26]:
len(candidate_sites)

6242

In [27]:
candidate_sites[5000]

'www.contracthound.com'

In [28]:
# make list of already-RA-examined sites.  AGAIN, ONLY GOES UP TO 11-9, LATER SITES NEED TO BE CHOPPED LATER
sites_to_exclude = [url.urlparse(x).netloc for x in oldurls + recenturls]

AttributeError: 'float' object has no attribute 'decode'

In [29]:
# why in the hell is there a float in there?  Something probably got a NaN in pandas because it was blank in the csv. 
sites_to_exclude = [url.urlparse(x).netloc for x in oldurls + [y for y in recenturls if type(y) is not float]]

In [30]:
len(sites_to_exclude)

917

In [31]:
sites_to_scrape = list(set(candidate_sites) - set(sites_to_exclude))

In [32]:
len(sites_to_scrape)

5852

In [34]:
with open("sites_to_scrape.json", "w") as s2s:
    json.dump(sites_to_scrape, s2s)